<a href="https://colab.research.google.com/github/maragraziani/interpretAI_DigiPath/blob/main/concept_attribution_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
EXPERIMENT_TYPE = 'RCV_VGG_PCAM'

# Concept attribution
... description

In [ ]:
!pip install rcvtool==0.1.2

In [2]:
#!pip install tensorflow==1.8.0
#!pip install keras==2.1.6
!pip install tensorflow
import tensorflow.keras
import sys
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD, Adadelta
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras import metrics
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import time
import sys
import shutil
import numpy as np
from keras.preprocessing import image
import cv2

In [3]:
tensorflow.keras.backend.clear_session()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import numpy as np
global new_folder
folder_name=EXPERIMENT_TYPE
new_folder = './'+ folder_name #new_folder
if not os.path.exists(new_folder):
    os.mkdir(new_folder)
global error_log
error_log=open(new_folder+'/ERR.log', 'w')
BATCH_SIZE = 32

# SAVE FOLD
f=open(new_folder+"/seed.txt","w")
seed=0
print(seed)
f.write(str(seed))
f.close()

# SET SEED
np.random.seed(seed)
#tf.set_random_seed(seed)

0


In [ ]:
# STAIN NORMALIZATION FUNCTIONS
def get_normalizer(patch, save_folder=''):
    normalizer = ReinhardNormalizer()
    normalizer.fit(patch)
    np.save('{}/normalizer'.format(save_folder),normalizer)
    np.save('{}/normalizing_patch'.format(save_folder), patch)
    print('Normalisers saved to disk.')
    return normalizer

def normalize_patch(patch, normalizer):
    return np.float64(normalizer.transform(np.uint8(patch)))
    
# BATCH GENERATORS
import sys
sys.path.append('/content/drive/MyDrive/CNNinterpret/')
from normalizers import *
global normalizer
normalizer=get_normalizer(np.load('/content/drive/MyDrive/CNNinterpret/normalizing_patch.npy'))
def get_batch_data(patch_list, labels, batch_size=32):
    num_samples=len(patch_list)
    while True:
        for offset in range(0,num_samples, batch_size):
            batch_x = []
            batch_y = []
            batch_samples=patch_list[offset:offset+batch_size]
            for patch_id in range(len(batch_samples)):
                #print(len(batch_samples))
                patch=patch_list[patch_id]
                patch=normalize_patch(patch, normalizer)
                # VGG input is fixed to 224,224 for finetuning imagenet weights
                patch=cv2.resize(patch, (224,224)) 
                patch=keras.applications.vgg16.preprocess_input(patch)
                label=labels[patch_id]
                batch_x.append(patch)
                batch_y.append(label)
            
            batch_x=np.asarray(batch_x, dtype=np.float32)/255.
            batch_y=np.asarray(batch_y, dtype=np.float32)
            yield batch_x, batch_y

Using brightness standardization
Normalisers saved to disk.


In [6]:
!wget https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz
!mv camelyonpatch_level_2_split_valid_x.h5.gz /content/drive/MyDrive/CNNinterpret/
!gunzip /content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_x.h5.gz


--2021-02-09 20:01:32--  https://zenodo.org/record/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 805965320 (769M) [application/octet-stream]
Saving to: ‘camelyonpatch_level_2_split_valid_x.h5.gz’

camelyonpatch_level 100%[===================>] 768.63M  5.50MB/s    in 2m 15s  

2021-02-09 20:03:49 (5.69 MB/s) - ‘camelyonpatch_level_2_split_valid_x.h5.gz’ saved [805965320/805965320]

gzip: /content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_x.h5 already exists; do you wish to overwrite (y or n)? y
y


In [7]:
import h5py as hd
y_test = np.asarray(hd.File('/content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_y.h5', 'r')['y'][:])
x_test = np.asarray(hd.File('/content/drive/MyDrive/CNNinterpret/camelyonpatch_level_2_split_valid_x.h5', 'r')['x'][:])
y_test=y_test[:].ravel()

In [ ]:
!wget https://zenodo.org/record/4322725/files/weights.h5
!mv weights.h5 /content/drive/MyDrive/CNNinterpret/

--2021-01-08 13:49:06--  https://zenodo.org/record/4322725/files/weights.h5
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 537118144 (512M) [application/octet-stream]
Saving to: ‘weights.h5’

weights.h5          100%[===================>] 512.24M  2.42MB/s    in 2m 43s  

2021-01-08 13:51:51 (3.14 MB/s) - ‘weights.h5’ saved [537118144/537118144]

--2021-01-08 13:59:23--  https://zenodo.org/record/4322725/files/weights.h5
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 537118144 (512M) [application/octet-stream]
Saving to: ‘weights.h5’

weights.h5          100%[===================>] 512.24M  14.4MB/s    in 2m 0s   

2021-01-08 14:01:25 (4.26 MB/s) - ‘weights.h5’ saved [537118144/537118144]



In [10]:
#Let's load the VGG weights finetuned on the histopathology dataset.
base_model=tensorflow.keras.applications.VGG16()
predictions=tensorflow.keras.layers.Dense(1,activation='sigmoid')(base_model.layers[-2].output)
vgg_histo_model = tensorflow.keras.Model(base_model.input, predictions)
vgg_histo_model.load_weights('/content/drive/MyDrive/CNNinterpret/weights.h5')
vgg_histo_model.compile(loss=tensorflow.keras.losses.binary_crossentropy, optimizer=tensorflow.keras.optimizers.SGD(lr=1e-4), metrics=['accuracy'])

In [ ]:
vgg_histo_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
# Let's extract the concept measures
import rcvtool as rcv
NSAMPLES=500
test_generator = get_batch_data(x_test[:NSAMPLES], y_test[:NSAMPLES])

layer = 'block5_conv3'
all_cm=np.zeros(NSAMPLES)
all_acts=np.zeros((NSAMPLES, 512))

i=0
for step in range(NSAMPLES//BATCH_SIZE):
  x_b, y_b = test_generator.__next__()
  cm_b=[]
  for x in x_b:
    cm_b.append(rcv.get_texture_measure(x, mtype='contrast')) 
  all_cm[i:i+len(cm_b)]=cm_b
  acts_b=rcv.get_batch_activations(vgg_histo_model, 'block5_conv3',x_b,pooling='AVG')
  all_acts[i:i+len(acts_b)]=acts_b
  i+=len(x_b)


ModuleNotFoundError: ignored

In [ ]:
rcv.get_rcv(all_acts,all_cm,type='global linear', evaluation=True, verbose=True)

Global linear regression under euclidean assumption
Random state:  1
R2:  0.7901418891157806
TEST mse: 1799.4093439089452, r2: 0.8138909797681035
